In [1]:
# default_exp scraper

# Scraper

> This module is used to scrape https://www.point2homes.com for all land available on Vancouver Island.

In [2]:
#hide
from nbdev.showdoc import *
from lxml.etree import HTML
from selenium import webdriver
from selenium.webdriver.common.by import By
import xmltodict as xd
from time import sleep
from lxml import etree
import pandas as pd
import re

In [ ]:
def get_text(xpath):
    return re.sub('<[^<]+?>', '', etree.tostring(xpath, method='html', with_tail=False).decode('ascii')).replace('\n', '').replace('  ', '')

In [110]:
def extract_properties(l):
    price = l.xpath(".//div[@class='price']")[0].text.replace('\n', '')

    name = get_text(l.xpath(".//h3[@class='name']")[0])

    location = l.xpath(".//div[@class='location']")[0].getchildren()[1].text

    size = get_text(l.xpath(".//div[@class='accommodation']")[0])

    description = get_text(l.xpath(".//div[@class='description-text']")[0])

    link = l.xpath(".//a")[0].attrib['href']

    properties = {
        'price': price,
        'name': name,
        'location': location,
        'size': size,
        'description': description,
        'link': link,
    }
    return properties

In [111]:
def read_land_column(lands):
    land_list = []
    for land in lands:
        land_list.append(extract_properties(land))
    return land_list

In [156]:
url = "https://www.thailand-property.com/land-for-sale?sort=min_price&page=1"
browser = webdriver.Firefox()
browser.get(url)

In [157]:
land_data = []
while(1):
    # Read Page

    page_source = HTML(browser.page_source)

    # Extract Land Column
    lands = page_source.xpath("//div[@class='description-block']")

    # Append data
    land_data = land_data + read_land_column(lands)
    
    if not lands:
        print("Likely at the last page. Ending scraping. Goodbye.")
        break
    
    # Report in.
    print(f"Successfully Scraped: {len(lands)}")

    # Go to next page
    url = browser.current_url.split('=')
    url[-1] = str(int(browser.current_url.split('=')[-1])+1)
    url = '='.join(url)
    browser.get(url)
    sleep(12)

Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25
Successfully Scraped: 25


KeyboardInterrupt: 

In [18]:
def extract_numbers(s):
    return float(''.join(re.findall("[-+]?\d*\.\d+|\d+", s)) or 0)

In [224]:
df = pd.DataFrame(land_data)
df = df[df['size'].str.contains('m2')].copy(deep=True)

df['price_float (USD)'] = df['price'].apply(extract_numbers) * 0.031 # Baht to USD
df['lot_size_float (ACRE)'] = df['size'].apply(lambda x: extract_numbers(x.split('m2')[0])) / 4046.86 # Square meters to acres

df['Dollar per Acre'] = df['price_float (USD)'] / df['lot_size_float (ACRE)']
df = df.sort_values('Dollar per Acre')
df['Dollar per Acre'] = df['Dollar per Acre'].apply(lambda x: str(round(x, 2)))

In [225]:
df

,price,name,location,size,description,link,price_float (USD),lot_size_float (ACRE),Dollar per Acre
969,"฿ 4,000,000",Exotic Land Plot with Feature Pond.,"Hua Hin, Prachuap Khiri Khan","1,920,000 m2/ 1,200 Rai",Beautiful piece of Land that can have its own ...,https://www.thailand-property.com/land-for-sal...,124000.0,474.441913,261.36
6,"฿ 350,000","Land for sale in Pak Chong, Ratchaburi","Chom Bueng, Ratchaburi","133,236 m2/ 83 Rai 1 Ngan 9 Sq.W.","Land for sale located at Pak Chong, Chom Bueng.",https://www.thailand-property.com/land-for-sal...,10850.0,32.923303,329.55
1135,"฿ 5,000,000",Land for with Resort Building for Sale or Ren...,"Mae On, Chiang Mai","6 7 2,952 m2/ 1 Rai 3 Ngan 38 Sq.W.",A plot of Land of 738 square wah (2952 square ...,https://www.thailand-property.com/6-bedroom-la...,155000.0,166.289914,932.11
33,"฿ 600,000",Land for sale 50 rai in HuaHin.,"Hua Hin, Prachuap Khiri Khan","80,000 m2/ 50 Rai","Land for sale 50 rai, 6.0 K / 1 rai Land tota...",https://www.thailand-property.com/land-for-sal...,18600.0,19.768413,940.89
397,"฿ 2,100,000","2 Bedroom Land for sale in Phak Kha, Sa Kaeo","Watthana Nakhon, Sa Kaeo","2 1 1,296 m2","Land for sale of 1,296sqm, consisting of 2 bed...",https://www.thailand-property.com/2-bedroom-la...,65100.0,52.212333,1246.83
...,...,...,...,...,...,...,...,...,...
54,"฿ 800,000",Rama 9 Land for sale.,"Huai Khwang, Bangkok",4 m2,LandUtility space 4 Rai&#160;Land in Rama9 are...,https://www.thailand-property.com/land-for-sal...,24800.0,0.000988,25090532.0
570,"฿ 2,640,000","Land for sale in Thung Khru, Bangkok","Thung Khru, Bangkok",8 m2/ 2 Sq.W.,"Land for sale located at Thung Khru, Bangkok.",https://www.thailand-property.com/land-for-sal...,81840.0,0.001977,41399377.8
701,"฿ 3,000,000",Superb Sea View Land Plots for Sale in Peacef...,"Ko Samui, Surat Thani",9 m2/ 2.25 Sq.W.,"This Superb Sea View Land for Sale in Lamai, i...",https://www.thailand-property.com/land-for-sal...,93000.0,0.002224,41817553.33
975,"฿ 4,000,000",Premium Sea view Land by Bang Por Beach.,"Ko Samui, Surat Thani",7 m2/ 1.75 Sq.W.,This superb sunset sea view Land is for sale i...,https://www.thailand-property.com/land-for-sal...,124000.0,0.001730,71687234.29


In [226]:
df.head(100).to_csv('ThailandTop100.csv', index=False)

In [252]:
df['Dollar per Acre'] = df['Dollar per Acre'].apply(float)

df_regions = df.head(100)[df.head(100)['location'].str.contains(', ')].copy(deep=True)

df_regions['region'] = df_regions['location'].apply(lambda x: x.split(',')[1])

In [253]:
df_regions

,price,name,location,size,description,link,price_float (USD),lot_size_float (ACRE),Dollar per Acre,region
969,"฿ 4,000,000",Exotic Land Plot with Feature Pond.,"Hua Hin, Prachuap Khiri Khan","1,920,000 m2/ 1,200 Rai",Beautiful piece of Land that can have its own ...,https://www.thailand-property.com/land-for-sal...,124000.0,474.441913,261.36,Prachuap Khiri Khan
6,"฿ 350,000","Land for sale in Pak Chong, Ratchaburi","Chom Bueng, Ratchaburi","133,236 m2/ 83 Rai 1 Ngan 9 Sq.W.","Land for sale located at Pak Chong, Chom Bueng.",https://www.thailand-property.com/land-for-sal...,10850.0,32.923303,329.55,Ratchaburi
1135,"฿ 5,000,000",Land for with Resort Building for Sale or Ren...,"Mae On, Chiang Mai","6 7 2,952 m2/ 1 Rai 3 Ngan 38 Sq.W.",A plot of Land of 738 square wah (2952 square ...,https://www.thailand-property.com/6-bedroom-la...,155000.0,166.289914,932.11,Chiang Mai
33,"฿ 600,000",Land for sale 50 rai in HuaHin.,"Hua Hin, Prachuap Khiri Khan","80,000 m2/ 50 Rai","Land for sale 50 rai, 6.0 K / 1 rai Land tota...",https://www.thailand-property.com/land-for-sal...,18600.0,19.768413,940.89,Prachuap Khiri Khan
397,"฿ 2,100,000","2 Bedroom Land for sale in Phak Kha, Sa Kaeo","Watthana Nakhon, Sa Kaeo","2 1 1,296 m2","Land for sale of 1,296sqm, consisting of 2 bed...",https://www.thailand-property.com/2-bedroom-la...,65100.0,52.212333,1246.83,Sa Kaeo
...,...,...,...,...,...,...,...,...,...,...
703,"฿ 3,096,000","Land for sale in Pa Sakae, Suphan Buri","Doem Bang Nang Buat, Suphan Buri","17,258.4 m2/ 10 Rai 3 Ngan 14.6 Sq.W.","Land for sale located at Pa Sakae, Doem Bang N...",https://www.thailand-property.com/land-for-sal...,95976.0,4.264640,22505.07,Suphan Buri
187,"฿ 1,400,000","Land for sale in Ban Fang, Khon Kaen","Ban Fang, Khon Kaen","7,768 m2/ 4 Rai 3 Ngan 42 Sq.W.","Land for sale located at Ban Fang, Khon Kaen.",https://www.thailand-property.com/land-for-sal...,43400.0,1.919513,22609.90,Khon Kaen
485,"฿ 2,500,000",Cheap Land for sale Adjacent to T-Park Indust...,"Si Racha, Chonburi","13,828 m2/ 8 Rai 2 Ngan 57 Sq.W.","Land for sale in purple, striped, adjacent to ...",https://www.thailand-property.com/land-for-sal...,77500.0,3.416970,22680.91,Chonburi
339,"฿ 1,900,000","Land for sale in Ban Khuan, Chumphon","Lang Suan, Chumphon","10,368 m2","Land for sale of 10,368sqm located at Ban Khua...",https://www.thailand-property.com/land-for-sal...,58900.0,2.561986,22989.97,Chumphon


In [263]:
a = df_regions.groupby('region').sum()[['price_float (USD)', 'lot_size_float (ACRE)']]

b = df_regions.groupby('region').count()[['location']]

c = df_regions.groupby('region').mean()[['Dollar per Acre']]

info = a.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)[['location','price_float (USD)', 'lot_size_float (ACRE)', 'Dollar per Acre']]

info.columns = ['Discounted Properties', 'Total Price (USD)', 'Total Size (ACRE)', 'Mean Dollar Per Acre on Discounted Properties']

In [273]:
info

,Discounted Properties,Total Price (USD),Total Size (ACRE),Mean Dollar Per Acre on Discounted Properties
region,,,,
Amnat Charoen,1,49600.000,3.162946,15681.580000
Chanthaburi,1,93000.000,4.513129,20606.550000
Chiang Mai,3,383625.000,180.918539,10802.040000
Chiang Rai,2,116808.000,14.898464,11034.570000
Chonburi,13,1090270.000,179.298518,10388.151538
Chumphon,1,58900.000,2.561986,22989.970000
Kamphaeng Phet,1,46500.000,3.953683,11761.190000
Khon Kaen,8,574324.600,97.347573,12109.157500
Lamphun,1,155000.000,8.556560,18114.760000


In [283]:
df.head()

,price,name,location,size,description,link,price_float (USD),lot_size_float (ACRE),Dollar per Acre
969,"฿ 4,000,000",Exotic Land Plot with Feature Pond.,"Hua Hin, Prachuap Khiri Khan","1,920,000 m2/ 1,200 Rai",Beautiful piece of Land that can have its own ...,https://www.thailand-property.com/land-for-sal...,124000.0,474.441913,261.36
6,"฿ 350,000","Land for sale in Pak Chong, Ratchaburi","Chom Bueng, Ratchaburi","133,236 m2/ 83 Rai 1 Ngan 9 Sq.W.","Land for sale located at Pak Chong, Chom Bueng.",https://www.thailand-property.com/land-for-sal...,10850.0,32.923303,329.55
1135,"฿ 5,000,000",Land for with Resort Building for Sale or Ren...,"Mae On, Chiang Mai","6 7 2,952 m2/ 1 Rai 3 Ngan 38 Sq.W.",A plot of Land of 738 square wah (2952 square ...,https://www.thailand-property.com/6-bedroom-la...,155000.0,166.289914,932.11
33,"฿ 600,000",Land for sale 50 rai in HuaHin.,"Hua Hin, Prachuap Khiri Khan","80,000 m2/ 50 Rai","Land for sale 50 rai, 6.0 K / 1 rai Land tota...",https://www.thailand-property.com/land-for-sal...,18600.0,19.768413,940.89
397,"฿ 2,100,000","2 Bedroom Land for sale in Phak Kha, Sa Kaeo","Watthana Nakhon, Sa Kaeo","2 1 1,296 m2","Land for sale of 1,296sqm, consisting of 2 bed...",https://www.thailand-property.com/2-bedroom-la...,65100.0,52.212333,1246.83


In [285]:
df['Dollar per Acre'].mean() * 0.01

8680.923866278028

In [286]:
df_discounted = df[df['Dollar per Acre'] < df['Dollar per Acre'].mean() * 0.01]

In [288]:
len(df_discounted)

40

In [289]:
len(df)

1115

In [284]:
df['Dollar per Acre'].mean()

868092.3866278027

### We can filter by discount per region, that might be more interesting.
We can also make a panel widget that allows us to slide our discount threshold.
Additionally, we can render a map showing the regions. We can encode dollar per acre per property on the map.

## Response to Feedback Top 60 in Surat Thani

In [22]:
def process(df):
    # Get Meters Squared
    df = df[df['size'].notna()]
    df = df[df['size'].str.contains('m2')].copy(deep=True)

    # Convert to Dollars and Acres
    df['price_float (USD)'] = df['price'].apply(extract_numbers) * 0.031 # Baht to USD
    df['lot_size_float (ACRE)'] = df['size'].apply(lambda x: extract_numbers(x.split('m2')[0])) / 4046.86 # Square meters to acres

    # Compute Dollar per Acre
    df['Dollar per Acre'] = df['price_float (USD)'] / df['lot_size_float (ACRE)']
    df = df.sort_values('Dollar per Acre')
    df['Dollar per Acre'] = df['Dollar per Acre'].apply(lambda x: str(round(x, 2)))
    df['Dollar per Acre'] = df['Dollar per Acre'].apply(float)
    
    # Get regions
    df = df[df['location'].str.contains(', ')].copy(deep=True)
    df['region'] = df['location'].apply(lambda x: x.split(',')[1])
    
    return df

In [115]:
def get_info(df):
    a = df.groupby('region').sum()[['price_float (USD)', 'lot_size_float (ACRE)']]
    b = df.groupby('region').count()[['location']]
    c = df.groupby('region').mean()[['Dollar per Acre']]
    info = a.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)[['location','price_float (USD)', 'lot_size_float (ACRE)', 'Dollar per Acre']]
    info.columns = ['Discounted Properties', 'Total Price (USD)', 'Total Size (ACRE)', 'Mean Dollar Per Acre on Discounted Properties']

In [117]:
df

,price,name,location,size,description,link
0,"฿ 200,000","Land for sale in Tha Kham, Bangkok","Bang Khun Thian, Bangkok",51 m2/ 12.75 Sq.W.,"Land for sale located at Tha Kham, Bang Khun T...",https://www.thailand-property.com/land-for-sal...
1,"฿ 250,000",Large Land and Oil Palm Plantation.,"Bang Khan, Nakhon Si Thammarat",NaN,"Palm Plantation, Tha Rai SubdistrictLand area ...",https://www.thailand-property.com/land-for-sal...
2,"฿ 300,000","Land for sale in Kut Chap, Udon Thani","Kut Chap, Udon Thani",616 m2/ 1 Ngan 54 Sq.W.,"Land for sale located at Kut Chap, Udon Thani.",https://www.thailand-property.com/land-for-sal...
3,"฿ 320,000","Land for sale in Don Ko Ka, Chachoengsao","Bang Nam Priao, Chachoengsao",236 m2,"Land for sale of 236sqm located at Don Ko Ka, ...",https://www.thailand-property.com/land-for-sal...
4,"฿ 350,000","Land for sale in Kham Khuean Kaeo, Ubon Ratcha...","Sirindhorn, Ubon Ratchathani","1,548 m2","Land for sale of 1,548sqm located at Kham Khue...",https://www.thailand-property.com/land-for-sal...
...,...,...,...,...,...,...
1245,"฿ 5,500,000",the cheap Land sea view.,"Ko Samui, Surat Thani","1,600 m2/ 1 Rai",Location : Kao pra very near to chawengsize: 1...,https://www.thailand-property.com/land-for-sal...
1246,"฿ 5,500,000",Premium Sunrise Sea view Land on Lamai Hillsi...,"Ko Samui, Surat Thani",907 m2/ 2 Ngan 26.75 Sq.W.,These unique ocean-view Land plots for sale in...,https://www.thailand-property.com/land-for-sal...
1247,"฿ 5,500,000",S757: KOH SAMUI Land PLOT FOR SALE WITH 180 D...,"Ko Samui, Surat Thani","1,100 m2/ 2 Ngan 75 Sq.W.",Situated on the hillside in the region of Bang...,https://www.thailand-property.com/land-for-sal...
1248,"฿ 5,500,000",Land For Sale In Hua Hin Near New Development...,"Hua Hin, Prachuap Khiri Khan",800 m2/ 2 Ngan,New developments of Homes and condos are in th...,https://www.thailand-property.com/land-for-sal...


In [116]:
get_info(df)

KeyError: 'region'

In [23]:
df = pd.read_csv('Thailand_raw.csv')

In [26]:
pdf = process(df)

In [32]:
pdf[pdf['region'] == ' Surat Thani']

,price,name,location,size,description,link,price_float (USD),lot_size_float (ACRE),Dollar per Acre,region
192,"฿ 1,400,000",Maenam Chanote Land For Sale Koh Samui.,"Ko Samui, Surat Thani","27,800 m2/ 17 Rai 1 Ngan 50 Sq.W.",Maenam Chanote Land For Sale Koh Samui &#8211;...,https://www.thailand-property.com/land-for-sal...,43400.0,6.869524,6317.76,Surat Thani
971,"฿ 4,000,000",Exclusive Sea View Land For Sale in Bangpor.,"Ko Samui, Surat Thani","51,200 m2/ 32 Rai",This stunning sea view Land for sale in Koh Sa...,https://www.thailand-property.com/land-for-sal...,124000.0,12.651784,9800.99,Surat Thani
1164,"฿ 5,000,000",Peaceful Plot of Freehold Land by Samui Golf ...,"Ko Samui, Surat Thani","22,504 m2/ 14 Rai 26 Sq.W.","This freehold Land is for sale in Maenam, Koh ...",https://www.thailand-property.com/land-for-sal...,155000.0,5.560855,27873.41,Surat Thani
668,"฿ 3,000,000","Land for sale in Wiang, Surat Thani","Chaiya, Surat Thani","12,304 m2","Land for sale of 12,304sqm located at Wiang, C...",https://www.thailand-property.com/land-for-sal...,93000.0,3.040382,30588.26,Surat Thani
640,"฿ 2,948,000",180 Degree Sunset Samui Sea view Land Plots F...,"Ko Samui, Surat Thani","7,450 m2/ 4 Rai 2 Ngan 62.5 Sq.W.",These panoramic sunset sea-view Land plots for...,https://www.thailand-property.com/land-for-sal...,91388.0,1.840933,49642.21,Surat Thani
...,...,...,...,...,...,...,...,...,...,...
1062,"฿ 4,500,000",Premium Sea View Land Plot for Sale in Plai L...,"Ko Samui, Surat Thani",400 m2/ 1 Ngan,This premium Land for sale in Koh Samui in loc...,https://www.thailand-property.com/land-for-sal...,139500.0,0.098842,1411342.43,Surat Thani
1163,"฿ 5,000,000",S1148: KOH SAMUI Land PLOT FOR SALE WITH BREA...,"Ko Samui, Surat Thani",400 m2/ 1 Ngan,A rare opportunity to purchase a Koh Samui Lan...,https://www.thailand-property.com/land-for-sal...,155000.0,0.098842,1568158.25,Surat Thani
1157,"฿ 5,000,000",400 sq m Sea view Land.,"Ko Samui, Surat Thani",400 m2/ 1 Ngan,This sea view plot is in one of the best locat...,https://www.thailand-property.com/land-for-sal...,155000.0,0.098842,1568158.25,Surat Thani
701,"฿ 3,000,000",Superb Sea View Land Plots for Sale in Peacef...,"Ko Samui, Surat Thani",9 m2/ 2.25 Sq.W.,"This Superb Sea View Land for Sale in Lamai, i...",https://www.thailand-property.com/land-for-sal...,93000.0,0.002224,41817553.33,Surat Thani


In [34]:
pdf['Dollar per Acre'].median()

313631.65

In [183]:
import param as pm
import panel as pn
import hvplot.pandas
pn.extension()

def get_info(df):
    a = df.groupby('region').sum()[['price_float (USD)', 'lot_size_float (ACRE)']]
    b = df.groupby('region').count()[['location']]
    c = df.groupby('region').mean()[['Dollar per Acre']]
    info = a.merge(b, left_index=True, right_index=True).merge(c, left_index=True, right_index=True)[['location','price_float (USD)', 'lot_size_float (ACRE)', 'Dollar per Acre']]
    info.columns = ['Properties For Sale', 'Total Price (USD)', 'Total Size (ACRE)', 'Mean Dollar Per Acre']
    info['Total Price (USD)'] = info['Total Price (USD)'].apply(lambda x: int(x))
    info['Total Size (ACRE)'] = info['Total Size (ACRE)'].apply(lambda x: round(x, 2))
    info['Mean Dollar Per Acre'] = info['Mean Dollar Per Acre'].apply(lambda x: round(x, 2))

    return info

In [184]:
get_info(pdf)

,Properties For Sale,Total Price (USD),Total Size (ACRE),Mean Dollar Per Acre
region,,,,
Amnat Charoen,1,49600,3.16,15681.58
Ang Thong,1,18600,0.40,47044.75
Bangkok,78,8491718,11.57,1995052.78
Bueng Kan,1,55800,0.12,470447.48
Buriram,3,212350,2.69,152593.35
Chachoengsao,6,507694,5.26,148121.95
Chaiyaphum,2,108500,1.51,80367.72
Chanthaburi,17,1604250,27.02,141934.91
Chiang Mai,102,8290615,237.94,3236632.87


In [194]:
class DiscountExplorer(pm.Parameterized):
    df = pm.DataFrame(precedence=-1)
    discount = pm.Number(0.5, bounds=(0,1), step=0.01)
    region = pm.ObjectSelector(default='ALL', objects=['ALL'])
    def __init__(self, df, **params):
        super(DiscountExplorer, self).__init__(**params)
        self.df = df
        self.param['region'].objects += sorted(df['region'].unique())
        
    @pm.depends('discount', 'region')
    def discount_view(self):
        if self.region == 'ALL':
            df = self.df
        else:
            df = self.df[self.df['region'] == self.region]
        average_price = df['Dollar per Acre'].mean()
        discount_price = average_price * (1 - self.discount)
        discounted = df[df['Dollar per Acre'] <= discount_price]
        stats = {
            "Average DPA": round(average_price, 2),
            "Discount Percent": self.discount,
            "Discount DPA": round(discount_price,2),
            "Showing": str(len(discounted)) + '/' + str(len(df))
                }
        return pn.Row(discounted.hvplot.table(width=1200, height=800), stats)
    
    def summary_view(self):
        return get_info(self.df).reset_index().hvplot.table(width=1200, height=800)

In [195]:
de = DiscountExplorer(pdf)

In [196]:
pd.set_option('display.max_colwidth', -1)

In [199]:
pn.Tabs(
    ('Land', 
     pn.Tabs(
         ('Region Summary', pn.Pane(de.summary_view)),
         ('Discounts', pn.Column(de, de.discount_view)),
     )), 
     ('Commercial', 
      pn.Tabs(
          ('Region Summary', pd.DataFrame()),
          ('Discounts', pd.DataFrame()),
      ),
     )).show()

Launching server at http://localhost:37493


In [161]:
pn.Pane(de.summary_view)

ParamMethod(method)